## Selectively stretch and decompress segments
<p>The goal is to enable selectively staying on one place for a longer time. This can be accomplished by selecting a segment of an array to double in size. </P>
<p>The assumption is that we will take the segmented chorale numpy arrays in directory segmented_chorales, and load them into this notebook and mess with them. This notebook will no longer need to synthesize arrays using the model, since that is done in the notebeeks called coconet_incremental_synthesis.<p>

In [ ]:
import numpy as np
import copy
import mido
import time
from midi2audio import FluidSynth
from IPython.display import Audio, display
import os
import muspy
import piano as p
import selective_stretching_codes as s
import samples_used as su
import subprocess
from numpy.random import default_rng
rng = np.random.default_rng()
soundfont = '../font.sf2' # you will need to download this from location specified in the github README.md
CSD_FILE = 'goldberg_aria1.csd'
LOGNAME = 'goldberg5.log'

## See if you can load one of the segmented fantasy chorales from the store of numpy arrays

Here's what this cell does:

- Load the numpy array, a (7, 16, 32) structure created by the coconet-model. Each of the 7 are independent 32 1/16th note sub-chorales based on taking an actual Bach chorale, BWV 180 'Schmücke dich, o liebe Seele'. A reminder that these are 16 voice chorales made by starting with three voices of Schmücke and using the coconet model to build the fourth voice. Then taking the new voice and two other existing voices and generating another. I kept doing that until I have four four voice chorales, or a 16 voice chorale, but one where each of the four have only a very limited knowledge of the other four chorales. It's sometimes a complex mess, but it holds somewhat together by the fact that they start with a real Bach chorale, and then slowly morph it into something artificial.
- Restore normal MIDI numbers
- expand and concatenate the segments back their original length. You have to pass a data structure to the expand_and_concatenate function that describes the current and desired length for all the segments. Their current lengths are always 32, since that is what comes out of the model. The desired length is what you would like to be the length of the segment. The function can only extend the length of the model by doubling the last 8 or more time_steps. It's doesn't have any other capabilities.
- transpose it back to its original key
- arpeggiate it
- make a midi from the arpeggiated chorale
- save it
- use Csound or MIDI player to generate a wav file.  

In [ ]:
# load the chorale from the midi file
# #  wake up                 0       1       2       3       4       5      6       7      
# expand_codes = np.array([[32,32],[32,32],[32,32],[32,32],[32,32],[32,32],[32,32],[32,32]])
# file_name = os.path.join('midi_files', 'Wake up, wake up, you sleepers.mid')
# SEGMENTS = 8 # wake up and dearest have 8 phrases and schmucke has 9 32 1/16th note phrases 
#
file_name = os.path.join('midi_files', 'schmucke.mid')
SEGMENTS = 9
# schmucke                0       1       2       3       4       5       6       7       8
expand_codes = np.array([[32,40],[32,40],[32,40],[32,40],[32,40],[32,40],[32,32],[32,32],[32,32]])
#
# file_name = os.path.join('midi_files', 'Dearest_Jesus.mid') 
# SEGMENTS = 6
# #  dearest                0       1       2       3       4       5      
# expand_codes = np.array([[32,32],[32,48],[32,32],[32,48],[32,32],[32,48]])
sample, root, mode = s.midi_to_input(file_name) # sample is time interval, voice
keys = ['C ','C#','D ','D#','E ','F ','F#','G ','G#','A ','A#','B ']
print(f'{file_name}, \n{keys[root]} {mode} transposed into C and then used to create the segments')
print(f'sample.shape: {sample.shape}')

In [ ]:
# play a midi file version of the chorale
s.quick_play(s.transpose_up_segment(sample.transpose(),root),3) # transpose from the key the model wants into the original root key

In [ ]:
# sample is a piano roll of pitches in 1/16th note intervals of dimension (240 time intervals, 4 voices, 1 pitch per time interval and voice)
# This cell divides the chorale into phrases at or above 32 1/16th note segments. 
print(f'sample.shape: {sample.shape}')
# Dearest Jesus does not require this concatenation
# this is for schmucke
sample = np.concatenate((sample, np.zeros((16,4))),axis=0) # Schmucke chorale needs a bit more at the end.
print(f'sample.shape: {sample.shape}')
seg_num = 0 # index into the segment array
max_seg_size = max(expand_codes[:,1])
min_seg_size = min(expand_codes[:,0])
max_pad = max_seg_size - min_seg_size
segment = np.zeros((SEGMENTS,4,max_seg_size),dtype=int)  # seg_num, voices, 1/16th note values
print(f'max_seg_size: {max_seg_size}, min_seg_size: {min_seg_size}, max_pad: {max_pad}')
pad_out = np.zeros((max_pad,4),dtype=int) # zeros to fill out the segment to max_seg_size
i = 0
current_location = 0
for seg in segment: 
    start = expand_codes[i,0] # the number of 1/16th notes that can be sent through the model
    end = expand_codes[i,1] # the number of 1/16th notes that the phrase provides
    print(f'segment: {i}. start: {start}, end: {end}, current_location: {current_location}')
    if end == max_seg_size: # no padding required
        transfer = sample[current_location:current_location + end]
    else: # need some padding and a concatenation
        pad_len = max_seg_size - end  # how much padding the phrases need to reach the maximum segment length
        pad_shape = pad_out[0:pad_len,0:4]
        transfer = np.concatenate((sample[current_location:current_location + end], pad_shape),axis=0)
    current_location += end
    segment[i] = transfer.transpose() 
    i += 1
print(f'segment.shape: {segment.shape}')
print(f'current_location at the end: {current_location}')
print(f'last segment first voice. segment[{i-1},0,:]: {segment[i-1,0,:]}')

In [ ]:
# just for Schmucke, we need to delete segments 2 & 3, since those are repeats, and reduce the SEGMENTS down to 7.
SEGMENTS = 7
print(f'expand_codes: {expand_codes}')
expand_codes = expand_codes[2:]
print(f'expand_codes: {expand_codes}')
print(f'segment.shape: {segment.shape}')
segment = segment[2:]
print(f'segment.shape: {segment.shape}')

In [ ]:
# some of the segments are longer than the 32 1/16th notes, so they have to be compressed. I do that by taking every other
# note from the end of the chorale, based on the sizes speficied in the array expand_codes.
max_seg_size = max(expand_codes[1])
min_seg_size = min(expand_codes[1])
print(f'max_seg_size: {max_seg_size}, min_seg_size: {min_seg_size}')
compress_segment = np.zeros((segment.shape[0],4,32),dtype=int) # match the input segment, but only 32 notes
i = 0
if max_seg_size > min_seg_size:
    for cur_seg in segment: 
        print(f'i: {i}')
        if expand_codes[i,1] == expand_codes[i,0]: # no expansion needed
            compress_segment[i] = cur_seg[:,:32]
        else:
            print(f'segment {i}. cur_seg.shape: {cur_seg.shape}')
            print(f'cur_seg: {cur_seg[:1,-24:]}')
            end = expand_codes[i,1]
            start = expand_codes[i,0] + (expand_codes[i,0] - end) # 32 + - 8 = 24
            print(f'compress from {cur_seg.shape[1]} to {expand_codes[0]}. Start at {start} take every other note')
            compress_segment[i] = s.compress_segment(cur_seg, start, expand_codes[i,1]) 
            print(f'compressed {expand_codes[i,1]} to {expand_codes[i,0]} and store it in compress_segment.shape: {compress_segment.shape}')
            print(f'compress_segment[{i},:1,-24:]: {compress_segment[i,:1,-24:]}')
        i += 1
else:
    compress_segment = segment 

sub_segment = compress_segment[:,:,:32] # This is only if you don't want to do compression, just truncation.
print(f'sub_segment.shape: {sub_segment.shape}') # it's now (segments, voices, notes)

In [ ]:
# Now expand the chorale back to it's uncompressed shape, concatenated into one long (voices,notes) array.
# It looses some 1/16 notes in the compressed section.
chorale = \
    s.expand_and_concatenate(sub_segment, expand_codes) # decompress 
print(f'chorale.shape: {chorale.shape}')  # now it's all one continuous (voices,notes) array
# now tranpose it back to the original key
chorale = s.transpose_up_segment(chorale,root) 
s.quick_play(chorale,3)

In [ ]:
# save the original chorale as written
print(f'sub_segment.shape: {sub_segment.shape}')
new_voices = sub_segment
filename = os.path.join('schmucke_chorales','chorale_HP800100.npy')
print(f'Original chorale. Saving to {filename}')
np.save(filename,new_voices)

In [ ]:
# I have three directories filled with numpy arrays of (segments, voices, notes) 
# each directory has one file that represents the chorale as written, but compressed down to no more than 32 notes per phrase
# I have to decompress them before playing them.
as_written_chorale = 'chorale_HP800100'
numpy_file = os.path.join('schmucke_chorales', as_written_chorale + '.npy')

chorale = np.load(numpy_file)
print(f'chorale.shape: {chorale.shape}')
chorale = \
    s.expand_and_concatenate(chorale, expand_codes) # decompress 
chorale = s.transpose_up_segment(chorale,root) 
print(f'chorale.shape: {chorale.shape}')
s.quick_play(chorale,3)

## Evaluate the chorales in segmented_chorale using muspy metrics. 

### Search for the "H" with high pitch class entropy.

This will load the chorales one at a time and run the muspy metrics against the finished decoded chorale. I don't put much stock in these metrics, except for the class entropy. That one gives extra value for violating classical theory rules. I like those with character.

This measure is derived from a paper by Wu & Yang: The Jazz Transformer on the front line: exploring the shortcomings of ai-composed music through quantitative measures.
    <a href="https://arxiv.org/pdf/2008.01307.pdf"> Shih-Lun Wu and Yi-Hsuan Yang</a>
    
<blockquote>
        5.1 Pitch Class Histogram Entropy
To gain insight into the usage of different pitches, we first
collect the notes appeared in a certain period (e.g., a bar)
and construct the 12-dimensional pitch class histogram
−→h ,
according to the notes’ pitch classes (i.e. C, C#, ..., A#, B),
normalized by the total note count in the period such that
P
i
hi = 1. Then, we calculate the entropy of
−→h :
H(
−→h ) = −
X
11
i=0
hi
log2
(hi). (2)
The entropy, in information theory, is a measure of “uncertainty” of a probability distribution [40], hence we adopt
it here as a metric to help assessing the music’s quality in
tonality. If a piece’s tonality is clear, several pitch classes
should dominate the pitch histogram (e.g., the tonic and
the dominant), resulting in a low-entropy
−→h ; on the contrary, if the tonality is unstable, the usage of pitch classes
is likely scattered, giving rise to an
−→h with high entropy</blockquote>

In [ ]:
# the next line will generate a report on the chorales. 
# You can select which voices to include, and you will get a different report.
# For example [(0,16]) would use all the voices
# the directory should contain only those with the same dimensions, such as all (6,16,32) for Dearest Jesus
# or all (7,16,32) for Schmucke
# sorting is done to have the highest class entropy at the top.
dirname = 'schmucke_chorales'
print(expand_codes)
metrics = s.print_chorale_metric_report(dirname,0,16,expand_codes)

## Looking for interesting places to stop and linger a while.

The chorales are interesting, but begin sound too conventional. As a composer, I like to sometimes take and existing piece and make my own variations on it. Bach did it, as did Brahms, Liszt, and many other composers. One technique that I have used in the past is to find expecially interesting parts of a simple piece and draw them out more, while keeping the overall forward moving structure of the piece. I call it the 'find an interesting place stop and linger a while' method of variations. It's just one method to create a variation. In this case, I call some functions that identify sections of the chorale that are especially interesting, as a measured by the number of notes that are not in the root key, in this case F major. I identify those, then stretch them out from whatever their current time steps to something much longer. 

# Preferred order of the manipulation layers:

1. Load the numpy array into a variable: <code>chorale = np.load(os.path.join('dearest_jesus_chorales','chorale_HP80029.npy')) + 30</code>
2. Set the expand codes based on the original chorale: <code>expand_codes = np.array([[32,32],[32,48],[32,32],[32,48],[32,32],[32,56]])</code>
3. Restore the original phrase lengths: <code>chorale = s.expand_and_concatenate(chorale[:,8:16,:], expand_codes)</code>
4. Restore the original key: <code>chorale = s.transpose_up_segment(chorale,root)</code>
5. Get the scores for each time_step: <code>scores = s.assign_scores_to_time_steps(chorale, root, mode)</code>
6. Determine the steps to expand and those to retain as is. You might want to do this once for each 4 voice chorale.
            <code>range_of_steps = s.find_challenging_time_steps(scores)</code>
            <code>range_in_tune = s.find_in_tune_time_steps(chorale, range_of_steps)</code>
7. Lengthed the challenging sections:<code>chorale, challenging_steps, in_tune_steps = s.expand_challenging_time_steps(chorale, range_of_steps, range_in_tune, high = 15) # where high is the maximum to expand</code>
8. Double the length of the chorale: <code>chorale = np.concatenate((chorale[:4,:], chorale[4:]),axis=1) </code>
9. Double the density from 4 to 8 voices: <code>chorale = np.concatenate((chorale,chorale),axis = 0)</code>
10. Arpeggiate it: <code>chorale = s.arpeggiate(chorale,rng.integers(low=0, high=2, size=(8,8), dtype=np.uint8),3)</code>
11. Play it: <code>s.piano_roll_to_csound(chorale,67,15,1,4,challenging_steps, zfactor=18)</code>

In [ ]:
file_stub = 'chorale_HP80035.npy' # Highest pitch entropy
dirname = 'schmucke_chorales'
print(f'root: {root}, mode: {mode}')
chorale = np.load(os.path.join(dirname,file_stub)) + 30
print(f'shape of chorale after loading: chorale.shape: {chorale.shape}')
chorale = s.expand_and_concatenate(chorale, expand_codes)
print(f'shape of chorale after restore original phrase lengths: chorale.shape: {chorale.shape}')
chorale = s.transpose_up_segment(chorale,root)

masks = np.array([[16,18,2,6,3],[16,18,4,4,2],[16,20,1,4,3],[16,20,2,2,2]]) #these are the values to pass to the arpeggiate function
# split the chorale into separate paths with 4 voices each
concat_chorale = np.empty((4,0),dtype=int)
concat_challenging_steps = np.empty((0,4),dtype=int)
previous_size = 0
for i in range(4): # 
    print(f'Start of expansion of section {i}',end='\t')
    sub_chorale = chorale[i*4:(i+1)*4,:] 
    print(f'i: {i} sub_chorale.shape: {sub_chorale.shape}')
    scores = s.assign_scores_to_time_steps(sub_chorale, root, mode)
    range_of_steps = s.find_challenging_time_steps(scores, sub_chorale) 
    range_in_tune = s.find_in_tune_time_steps(sub_chorale, range_of_steps)
    sub_chorale, challenging_steps, in_tune_steps = \
        s.expand_challenging_time_steps(sub_chorale, range_of_steps, range_in_tune, high = 10)
    print(f'challenging_steps.shape: {challenging_steps.shape}')
    print(f'shape of chorale after expansion of section {i}. sub_chorale.shape: {sub_chorale.shape}')
    sub_chorale = s.arpeggiate(sub_chorale,s.shaded_mask((masks[i,0],masks[i,1]),masks[i,2],masks[i,3]),masks[i,4])
    concat_chorale = np.concatenate((concat_chorale, sub_chorale),axis = 1) # add this chorale to the chain of chorales
    challenging_steps += previous_size # zero at first, the next steps are boosted before concatenation, all four indeciis are boosted
    previous_size += sub_chorale.shape[1] # then all the values are increased next time through
    concat_challenging_steps = np.concatenate((concat_challenging_steps,challenging_steps),axis=0)
    print(f'concat_challenging_steps.shape: {concat_challenging_steps.shape}')
    s.report_expansion(scores, range_of_steps, challenging_steps, sub_chorale, concat_challenging_steps, lines = 250)
                                                                            
print(f'shape of chorale after expansion, lengthening, and concatenation: concat_chorale.shape: {concat_chorale.shape}')
chorale = np.concatenate((concat_chorale,concat_chorale),axis = 0) # double the voices from 4 to 8.
chorale = s.octave_shift(s.octave_shift(chorale, 0.15, 45), 0.15, 33) # 15% of the time increase the octave of a note, clump the changes in groups of 33 and 45
print(f'shape of chorale after increasing voices: chorale.shape: {chorale.shape}')

In [ ]:
s.piano_roll_to_csound(chorale, 65, 13, 1.05, 3, concat_challenging_steps, zfactor=18)

In [ ]:
version = 36
!csound goldberg_aria1c.csd
!ls -lth /home/prent/Music/sflib/goldberg_aria1a-c.wav
!sox /home/prent/Music/sflib/goldberg_aria1a-c.wav save1.wav reverse
!sox save1.wav save2.wav silence 1 0.01 0.01
!sox save2.wav save1.wav reverse
!sox save1.wav /home/prent/Music/sflib/goldberg_aria1-t36.wav silence 1 0.01 0.01
!rm save1.wav
!rm save2.wav
!ls -lth /home/prent/Music/sflib/goldberg_aria1-t36.wav
!ffmpeg -y -i /home/prent/Music/sflib/goldberg_aria1-t36.wav\
    -b:a 320k /home/prent/Music/sflib/goldberg_aria1-t36.mp3
!cp /home/prent/Music/sflib/goldberg_aria1-t36.mp3 \
    /home/prent/Dropbox/Uploads/goldberg_aria1-t36.mp3
audio = Audio('/home/prent/Music/sflib/goldberg_aria1-t36.mp3')
display(audio)

In [ ]:
how_many = su.report_samples_used()
print(f'number of samples used in the last csound run {how_many}')